# Etude scientifique du PerceiverIO
### Sujet de Nicolas Ragot - Polytech Tours

**Etudiants :**
- Theo Boisseau (theo.boisseau@etu.univ-tours.fr)
- Sarah Denis (sarah.denis-2@etu.univ-tours.fr)

# Description:

Ce Notebook vient en complément du rapport **BOISSEAU_DENIS_Projet_PerceiverIO**.

Le but de cet exemple est d'aider à la compréhension de l'utilisation de l'outil de deep
 learning **PerceiverIO** créé par *DeepMind* et d'évaluer ses performances.

Les éléments fournis sont les suivants:
- jeu de données d'exemple (./data/exampleText.txt) sous la forme d'un fichier .txt, correspondant
 à un extrait du livre électronique libre de droits du Projet Gutenberg *History of the United States*
 par Charles A. Beard et Mary R. Beard et disponible à l'adresse
 https://www.digitalbook.io/txt/1/6/9/6/16960/16960.txt;
- fichier d'instances sérialisées des hyperparamètres du modèle, entrainés par *Deepmind* et
 disponible à l'adresse https://storage.googleapis.com/perceiver_io/language_perceiver_io_bytes.pickle;
- script pour l'évaluation des performances du modèle;

# Sommaire :
1. [Chargement des Modules et configuration du modèle](#1-bullet)
2. [Définition des fonctions pour utiliser le PerceiverIO](#2-bullet)
3. [Chargement des hyperparamètres et du DataSet, et pré-traitement](#3-bullet)
4. [Application de la solution en local](#4-bullet)
5. [Mise en forme des résultats](#5-bullet)

# 1. Chargement des Modules

In [9]:
import sys
from typing import Union
from IPython.display import display

# Haiku permet aux utilisateurs d'utiliser des modèles de POO tout en permettant un accès complet aux transformations de fonctions pures de JAX pour les reseaux neuronaux.
import haiku as hk
# JAX est concue pour le calcul numérique à haute performance, notamment pour la recherche en apprentissage automatique.
import jax
import jax.numpy as jnp
# NumPy ajoute un support pour les grands tableaux multidimensionnels et les matrices, ainsi qu'une grande collection de fonctions mathématiques pour opérer sur ces tableaux.
import numpy as np
# Pickle est un module permettant la (dé)sérialisation
import pickle

from perceiver import *
import random_masked_language as rml

In [10]:
#@title Model config
D_MODEL = 768
D_LATENTS = 1280
MAX_SEQ_LEN = 2048

encoder_config = dict(
    num_self_attends_per_block=26,
    num_blocks=1,
    z_index_dim=256,
    num_z_channels=D_LATENTS,
    num_self_attend_heads=8,
    num_cross_attend_heads=8,
    qk_channels=8 * 32,
    v_channels=D_LATENTS,
    use_query_residual=True,
    cross_attend_widening_factor=1,
    self_attend_widening_factor=1)

decoder_config = dict(
    output_num_channels=D_LATENTS,
    position_encoding_type='trainable',
    output_index_dims=MAX_SEQ_LEN,
    num_z_channels=D_LATENTS,
    qk_channels=8 * 32,
    v_channels=D_MODEL,
    num_heads=8,
    final_project=False,
    use_query_residual=False,
    trainable_position_encoding_kwargs=dict(num_channels=D_MODEL))

# The tokenizer is just UTF-8 encoding (with an offset)
tokenizer = bytes_tokenizer.BytesTokenizer()

# 2. Définition des fonctions pour utiliser le PerceiverIO

In [11]:
# Agrandit les tableaux inputs et input_mask pour qu'ils aient
# une taille de max_sequence_length, et les remplit avec des 0
def pad(max_sequence_length: int, inputs, input_mask):
  input_len = inputs.shape[1]
  assert input_len <= max_sequence_length
  pad_len = max_sequence_length - input_len
  padded_inputs = np.pad(
      inputs,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=tokenizer.pad_token)
  padded_mask = np.pad(
      input_mask,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=0)
  return padded_inputs, padded_mask

In [12]:
#@title Decoding Perceiver Model
def apply_perceiver(
    inputs: jnp.ndarray, input_mask: jnp.ndarray) -> jnp.ndarray:
  """Runs a forward pass on the Perceiver.

  Args:
    inputs: input bytes, an int array of shape [B, T]
    input_mask: Array of shape indicating which entries are valid and which are
      masked. A truthy value indicates that the entry is valid.

  Returns:
    The output logits, an array of shape [B, T, vocab_size].
  """
  assert inputs.shape[1] == MAX_SEQ_LEN
  # EXCEPTION A GERER : LE TEXTE EST TROP LONG, IL FAUT LE SEPARER EN DES ENSEMBLES DE
  # PHRASES DE MOINS DE MAX_SEQ_LEN CARACTRES

  # https://gdcoder.com/what-is-an-embedding-layer/
  # Une couche d'integration convertie l'input en un ensemble de vecteurs
  # d'integration dont les tailles sont optimisees pour le calcul

  # Creation de la couche d'integration selon le nombre de mots distincts dans
  # le training set et la dimension voulue des vecteurs d'integrations
  embedding_layer = hk.Embed(
      vocab_size=tokenizer.vocab_size,
      embed_dim=D_MODEL)
  # Conversion de l'input pour rentrer dans la 1ere couche (couche d'integration)
  embedded_inputs = embedding_layer(inputs)

  # Taille de la conversion de l'input
  batch_size = embedded_inputs.shape[0]

  # Parametres de construction des informations entrainables a propos de
  # la position des mots dans la phrase
  input_pos_encoding = perceiver.position_encoding.TrainablePositionEncoding(
      index_dim=MAX_SEQ_LEN, num_channels=D_MODEL)

  # Ajout des informations de position des mots a l'input
  embedded_inputs = embedded_inputs + input_pos_encoding(batch_size)
  # Initialisation du PerceiverIO
  perceiver_mod = perceiver.Perceiver(
      encoder=perceiver.PerceiverEncoder(**encoder_config),
      decoder=perceiver.BasicDecoder(**decoder_config))

  # Stockage dans output du resultat de l'execution du PerceiverIO a partir de la 1ere couche
  output_embeddings = perceiver_mod(
      embedded_inputs, is_training=False, input_mask=input_mask, query_mask=input_mask)

  # Redimensionnement et decodage de l'output
  logits = io_processors.EmbeddingDecoder(
      embedding_matrix=embedding_layer.embeddings)(output_embeddings)
  return logits

# La transformation de la fonction lui permettra plus tard de lui
# passer des parametres pre-enregistres
# input_pos_encoding sera notamment ecrase par celui pre-enregistre
apply_perceiver = hk.transform(apply_perceiver).apply

# 3. Chargement des hyperparamètres et du DataSet, et pré-traitement

In [13]:
# On deserialise les valeurs des hyperparametres du modele.

try: params
except NameError:
    try:
        print("Chargement des hyperparametres...")
        with open("./data/language_perceiver_io_bytes.pickle", "rb") as f:
            params = pickle.loads(f.read())
    except (FileNotFoundError, pickle.UnpicklingError):
        print("Aucun trouves. Telechargement des hyperparametres.")
        !wget -O ./data/language_perceiver_io_bytes.pickle \
                https://storage.googleapis.com/perceiver_io/language_perceiver_io_bytes.pickle

assert type(params).__name__ == 'FlatMapping'
print("Les hyperparametres ont ete charges.")

Chargement des hyperparametres...
Les hyperparametres ont ete charges.


In [14]:
# On prend un texte pour les tests
with open('./data/exampleText.txt', 'r') as f:
    inputs_str = f.read()
f.close()

if len(inputs_str) > 0:
    if len(inputs_str) < 100:
        print(inputs_str)
    else:
        print(f"Extrait: {inputs_str[0:100]}\n")

#pre-traitement des donnees
inputs_str = inputs_str.replace('\n', ' ')
inputs_str = inputs_str.replace('  ', ' ')
inputs_str = inputs_str.replace('--a', '')
inputs_str = inputs_str.replace('--', '-')
inputs_str = inputs_str.split(". ")
initial_len_inputs_str = len(inputs_str)
for sentenceIndex in range(len(inputs_str)):
    if len(inputs_str[sentenceIndex]) > MAX_SEQ_LEN:
        del inputs_str[sentenceIndex]
        sentenceIndex -= 1
    else:
        inputs_str[sentenceIndex] += '.'
print(f"Nombre de phrases trop longues : {initial_len_inputs_str-len(inputs_str)}")

dataSize = len(inputs_str)

Extrait: In the period between the landing of the English at Jamestown, Virginia,
in 1607, and the close of t

Nombre de phrases trop longues : 0


In [15]:
# Initialisation des variables pour chaque phrase
data = [
    {
        'input_str':inputs_str[sentenceIndex],
        'maskedwords':None,
        'maskedwordsIndexesInStr':None,
        'input_tokens':None,
        'inputs':None,
        'input_mask':None,
        'predicted_words':None,
        'efficiency':None
    } for sentenceIndex in range(dataSize)
]

try: percentage
except NameError: percentage = 20

for iterator in range(dataSize):
    data[iterator]['maskedwords'], maskedwordsIndexesInData = rml.chooseMaskedWords(data[iterator]['input_str'], percentage)
    data[iterator]['maskedwordsIndexesInStr'] = rml.findIndexes(data[iterator]['input_str'], maskedwordsIndexesInData)
    assert len(data[iterator]['maskedwordsIndexesInStr']) % 2 == 0
    data[iterator]['input_tokens'] = rml.inputWithMaskedWords(data[iterator]['input_str'], data[iterator]['maskedwordsIndexesInStr'])

print(f"Nombre de phrases a tester: {dataSize}")

Nombre de phrases a tester: 54


# 4. Application de la solution en local

In [16]:
for iterator in range(dataSize):
    # inputs est le tableau d'entiers avec une dimension supplementaire
    # c'est comme si on mettait le tableau inputs a l'interieur d'un nouveau tableau
    data[iterator]['inputs'] = data[iterator]['input_tokens'][None]

    # input_mask est l'equivalent unitaire de inputs :
    # pour toutes les valeurs d'inputs, on met un 1
    data[iterator]['input_mask'] = np.ones_like(data[iterator]['inputs'])

    data[iterator]['inputs'], data[iterator]['input_mask'] = pad(MAX_SEQ_LEN, data[iterator]['inputs'], data[iterator]['input_mask'])

assert [a * b for a,b in zip(data[0]['inputs'][0], data[0]['input_mask'][0])] == [a for a in data[0]['inputs'][0]]
print(f"Exemple de inputs: {data[0]['inputs']}")
print(f"Exemple de input_mask: {data[0]['input_mask']}")

Exemple de inputs: [[ 79 116  38 ...   0   0   0]]
Exemple de input_mask: [[1 1 1 ... 0 0 0]]


In [17]:
rng = jax.random.PRNGKey(1)  # Unused

for iterator in range(dataSize):
    # la prediction sous forme d'entiers est constituee des valeurs maximales
    # aux coordonnees du masque sur la 1ere dimension de out : ( type(out[0]) == int[] )
    out = apply_perceiver(params, rng=rng, inputs=data[iterator]['inputs'], input_mask=data[iterator]['input_mask'])
    data[iterator]['predicted_words'] = tokenizer.to_string(out[0].argmax(axis=-1))

    inputs_progression = int((iterator+1)/dataSize*100)
    if inputs_progression % 5 < 2:
        display(f"Progression dans les echantillons : {inputs_progression}%")

'Progression dans les echantillons : 1%'

'Progression dans les echantillons : 5%'

'Progression dans les echantillons : 11%'

'Progression dans les echantillons : 16%'

'Progression dans les echantillons : 20%'

...

'Progression dans les echantillons : 81%'

'Progression dans les echantillons : 85%'

'Progression dans les echantillons : 90%'

'Progression dans les echantillons : 96%'

'Progression dans les echantillons : 100%'

In [18]:
results = open('./results/analysis_results.txt', "w")

rng = jax.random.PRNGKey(1)  # Unused

for iterator in range(dataSize):
    predictedSentence = rml.stringWithNewWords(data[iterator]['input_str'], data[iterator]['predicted_words'], data[iterator]['maskedwordsIndexesInStr'])
    masked_words_predictions = rml.extractWordsByIndexes(data[iterator]['predicted_words'], data[iterator]['maskedwordsIndexesInStr'])

    results.write("-"+str(iterator+1)+"-\n")
    results.write("-Sentence:"+"\n")
    results.write(data[iterator]['input_str']+"\n")
    results.write("-Sentence predicted:"+"\n")
    results.write(predictedSentence+"\n")

    results.write("-Word-by-word prediction:"+"\n")
    for local_masked_word_prediction in masked_words_predictions:
        results.write("\t>"+local_masked_word_prediction+"\n")

    efficiency = data[iterator]['efficiency'] = \
        rml.computeEfficiency(data[iterator]['input_str'], predictedSentence, data[iterator]['maskedwordsIndexesInStr'])
    results.write("-Efficiency:"+"\n")
    if efficiency is not None:
        results.write('{:.2f}%'.format(efficiency * 100)+"\n\n")
    else:
        results.write("None"+"\n\n")

results.close()
print(f"Outputs disponibles dans {results.name}")

Outputs disponibles dans ./results/analysis_results.txt


# 5. Mise en forme des résultats

In [19]:
"""
efficiency_samples = [ ( data_iterator, efficiency ), ... ]

"""
efficiency_samples = []

model_efficiency = 0
for iterator in range(dataSize):
    efficiency = data[iterator]['efficiency']
    if efficiency is not None:
        efficiency_samples.append( (iterator, efficiency) )
        model_efficiency += efficiency
model_efficiency /= len(efficiency_samples)

print(f"Nombre de resultats: {len(efficiency_samples)}\n")
print(f"Efficacite moyenne du modele sur {f.name} avec un pourcentage de masquage de {percentage}: "
      f"{'{:.2f}%'.format(model_efficiency * 100)}")


Nombre de resultats: 51

Efficacite du modele sur ./data/exampleText.txt avec un pourcentage de masquage de 20: 47.86%
